In [1]:
import os
import numpy as np
import rasterio
from glob import glob
from tqdm import tqdm

In [2]:
def load_data(base_dir):

    s1_dir=glob(base_dir+'s1_validation'+'/'+'*.tif')
    s2_dir=[]
    lc_dir=[]
    for i in tqdm(range(len(s1_dir))):
        s1_filename=os.path.basename(s1_dir[i])
        former=s1_filename.split('_s1')[0]
        ID=s1_filename.split('_s1')[1]
        s2_dir.append(base_dir+'s2_validation/'+former+'_s2'+ID)
        lc_dir.append(base_dir + 'lc_validation/' + former + '_lc' + ID)

    s1_dir=np.array(s1_dir)
    s2_dir=np.array(s2_dir)
    lc_dir=np.array(lc_dir)

    return s1_dir,s2_dir,lc_dir

def read_data(f1,f2):
    
    with rasterio.open(f1) as patch:
        x1 = patch.read(list(range(1,3)))

    with rasterio.open(f2) as patch:
        x2 = patch.read(list(range(1,14)))

    return x1,x2

def clean(x1,x2):
    # s1
    x1[np.isnan(x1)] = 0
    # s2
    x2[np.isnan(x2)] = 0

    # s1_recommend
    x1[x1<-25]=-25
    x1[x1>0]=0
    # s2_recommend
    x2[x2 < 0] = 0
    x2[x2>10000]=10000

    return x1.astype('float16'),x2.astype('float16')

def norm(x1,x2):
    # input,x1:[-25,0],x2:[0,10000]
    h,w,c1=x1.shape
    h,w,c2=x2.shape
    x2 /= 10000 * 1.0
    x1 = (x1 + 25) / 25 * 1.0
    return x1,x2

lab_dict={1:1,2:1,3:1,4:1,5:1,6:2,7:2,8:3,9:3,10:4,11:5,12:6,14:6,13:7,15:8,16:9,17:10}

def process_label(y):
    C,H,W=y.shape
    y=y[[0],:,:]#1,h,w, simplified IGBP
    y=y.reshape(-1)
    y = list(map(lambda x: lab_dict[x], y))
    y=np.array(y)#list->array
    y=y.reshape(-1,H,W)
    y-=1 # start from 0
    return y

def filter_label(x2,y):

    x2[np.isnan(x2)] = 0

    x2[x2>10000]=10000
    x2[x2<0]=0

    x2 = x2.astype('float')

    R = x2[:, :, 3]
    G = x2[:, :, 2]
    B = x2[:, :, 1]
    Nir = x2[:, :, 7]  # TM4
    Mir = x2[:, :, 10]  # TM5
    SWir = x2[:, :, 11]  # TM7

    MSI = SWir / Nir
    NDWI = (G - Nir) / (G + Nir)
    NDVI = (Nir - R) / (Nir + R)
    NDBBI = (1.5 * SWir - (Nir + G) / 2.) / (1.5 * SWir + (Nir + G) / 2.)  # 归一化差值裸地与建筑用地指数
    BSI = ((Mir + R) - (Nir + B)) / ((Mir + R) + (Nir + B))  # 裸土指数
    NBI = R * SWir / Nir

    y_clean=y.copy()

    # Forest0,Shrubland1,bg_1-2,Grassland3,Wetlands4,Croplands5,Urban6,bg2-7,Barren8,water9
    
    # columns

    # 修正不符合要求的森林类
    y_clean[np.where((NDVI > 0.75) & (y != 0))] = 10
    # 修正不符合要求的灌木类
    y_clean[np.where((NDVI > 0.2) & (NDVI < 0.35) & (MSI > 1.5) & (y != 1))] = 10
    # 修正不符合要求的草地类
    y_clean[np.where((NDVI > 0.4) & (NDVI < 0.55) & (y != 3))] = 10
    # 修正不符合要求的湿地类
    y_clean[np.where((NDVI > 0.6) & (NDVI < 0.75) & (y != 4))] = 10
    # 修正不符合要求的农田类
    #y_clean[np.where((NDVI > 0.2) & (NDVI < 0.35) & (MSI > 1) & (MSI < 1.5) & (y != 5))] = 10
    # 修正不符合要求的建筑类
    #y_clean[np.where((NDVI > 0.2) & (NDVI < 0.35) & (MSI > 0.9) & (MSI < 1) & (y != 6))] = 10
    # 修正不符合要求的裸地类
    y_clean[np.where((NDVI > 0) & (NDVI < 0.15) & (y != 8))] = 10
    # 修正裸土建筑错分到其他类
    #y_clean[np.where((BSI > -0.4) & (NDVI < 0.15) & (y != 6) & (y != 8))] = 10
    # 修正不符合要求的水体类
    y_clean[np.where((NDWI > 0.) & (y != 9))] = 10

    # rows
    # forest

    # 修正其他类错标到森林
    y_clean[np.where((NDVI < 0.75) & (y == 0))] = 10

    # shrubland
    
    # savanna

    # 将热带草原标签修正为草地
    #y_clean[np.where((NDVI > 0.4) & (NDVI < 0.55) & (y == 2) & (np.sum(y == 9) < 2000))] = 3
    # 将热带草原标签修正为湿地
    y_clean[np.where((NDVI > 0.6) & (NDVI < 0.75) & (y == 2) & (np.sum(y == 9) > 10000))] = 4
    # 将热带草原标签修正为灌木
    y_clean[np.where((NDVI > 0.2) & (NDVI < 0.35) & (MSI > 1.5) & (y == 2))]=1
    # 将热带草原标签修正为裸地
    y_clean[np.where((NBI > 750) &(NDVI<0.2)& (NDVI>0) & (y == 2))] = 8
  
    # grassland

    # 将草地标签修正为湿地
    y_clean[np.where((NDVI > 0.6) & (NDVI < 0.75) & (y == 3) & (np.sum(y==9)>10000))] = 4

    # wetland
    # 将湿地修正为森林
    #y_clean[np.where((NDVI > 0.75) & (y == 4))] = 0

    y_clean[y_clean==2]=10
    y_clean[y_clean==7]=10

    return y_clean

## Loading & Preparing

In [3]:
base_dir = '/data/PublicData/DF2020/val/'
s1_pre, s2_pre, lc_pre=load_data(base_dir)

100%|██████████| 986/986 [00:00<00:00, 290440.60it/s]


In [4]:
s1_val=np.zeros([s1_pre.shape[0],256,256,2],dtype='float16')
s2_val=np.zeros([s1_pre.shape[0],256,256,10],dtype='float16')
y_val=np.zeros([s1_pre.shape[0],1,256,256],dtype='uint8')

In [5]:
for i in tqdm(range(s1_pre.shape[0])):
    s1_name=s1_pre[i]
    s2_name=s2_pre[i]
    y_name=lc_pre[i]
    x1,x2=read_data(s1_name,s2_name)
    x1=x1.transpose(1,2,0)#h,w,c
    x2=x2.transpose(1,2,0)
    x2_tmp=x2.copy()
    x2=x2[:,:,[1,2,3,4,5,6,7,10,11,12]]#remove b1,9,10
    
    x1,x2=clean(x1,x2)
    
    with rasterio.open(y_name) as patch:
        y = patch.read(list(range(1, 5)))
    y = process_label(y)
    y = filter_label(x2_tmp, y)
    x1, x2 = norm(x1, x2)
    
    s1_val[i,:,:,:]=x1
    s2_val[i,:,:,:]=x2
    y_val[i,:,:,:]=y

100%|██████████| 986/986 [03:11<00:00,  5.15it/s]


In [6]:
np.unique(y_val)

array([ 0,  1,  3,  4,  5,  6,  8,  9, 10], dtype=uint8)

merge

In [7]:
s_val=np.concatenate((s1_val,s2_val),axis=3)

In [8]:
s_val.shape

(986, 256, 256, 12)

In [9]:
s_val=s_val.reshape(-1,12)

In [10]:
s_val.shape

(64618496, 12)

In [11]:
y_val=y_val.reshape(-1)

In [12]:
y_val.shape

(64618496,)

filter

In [13]:
idx=y_val!=10

In [14]:
s_val=s_val[idx]
y_val=y_val[idx]

In [15]:
s_val.shape

(32933473, 12)

## Training

In [16]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

/data/di.wang/.conda/envs/pytorch/lib/python3.5/site-packages/xgboost/__init__.py:28: FutureWarning: Python 3.5 support is deprecated; XGBoost will require Python 3.6+ in the near future. Consider upgrading to Python 3.6+.
  FutureWarning)


In [18]:
X_trn,X_tes,y_trn,y_tes =train_test_split(s_val,y_val,test_size=0.4, random_state=0)

In [19]:
clf = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=10,
    tree_method='gpu_hist' , # THE MAGICAL PARAMETER
    learning_rate=0.1,
    n_gpus = 1 ,# -1表示使用所有GPU
    gpu_id = 3 , # 从GPU 1 开始
)

In [20]:
clf.fit(X_trn, y_trn, early_stopping_rounds=10, eval_metric="mlogloss",
        eval_set=[(X_tes, y_tes)])

[16:39:55] WARNING: /workspace/include/xgboost/generic_parameters.h:36: 
n_gpus: 
	Deprecated. Single process multi-GPU training is no longer supported.
	Please switch to distributed training with one process per GPU.
	This can be done using Dask or Spark.  See documentation for details.
[0]	validation_0-mlogloss:1.73194
Will train until validation_0-mlogloss hasn't improved in 10 rounds.
[1]	validation_0-mlogloss:1.49609
[2]	validation_0-mlogloss:1.31663
[3]	validation_0-mlogloss:1.17257
[4]	validation_0-mlogloss:1.05277
[5]	validation_0-mlogloss:0.95156
[6]	validation_0-mlogloss:0.86431
[7]	validation_0-mlogloss:0.78860
[8]	validation_0-mlogloss:0.72225
[9]	validation_0-mlogloss:0.66376
[10]	validation_0-mlogloss:0.61206
[11]	validation_0-mlogloss:0.56595
[12]	validation_0-mlogloss:0.52481
[13]	validation_0-mlogloss:0.48811
[14]	validation_0-mlogloss:0.45522
[15]	validation_0-mlogloss:0.42566
[16]	validation_0-mlogloss:0.39902
[17]	validation_0-mlogloss:0.37506
[18]	validation_0-mlog

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=3,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=nan, monotone_constraints=None,
       n_estimators=100, n_gpus=1, n_jobs=0, num_parallel_tree=1,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=None, subsample=1,
       tree_method='gpu_hist', validate_parameters=False, verbosity=None)

validation

In [21]:
y_pre=clf.predict(X_tes)
np.mean(y_pre==y_tes)

0.953671454348501

checking

In [22]:
print(np.unique(y_trn))
print(np.unique(y_tes))

[0 1 3 4 5 6 8 9]
[0 1 3 4 5 6 8 9]


saving

In [23]:
from sklearn.externals import joblib

In [24]:
joblib.dump(clf,'xgb_val_sa_shrb.pkl')

['xgb_val_sa_shrb.pkl']

## Predicting

In [25]:
hex_color_dict={10:'000000',0:'009900',1:'c6b044',2:'fbff13',3:'b6ff05',4:'27ff87',5:'c24f44',
                    6:'a5a5a5',7:'69fff8',8:'f9ffa4',9:'1c0dff'}

def Hex_to_RGB(str):
    r = int(str[0:2],16)
    g = int(str[2:4],16)
    b = int(str[4:6],16)
    return [r,g,b]

def DrawResult(labels, row, col):
    num_class = 10

    X_result = np.zeros((labels.shape[0], 3))
    for i in range(num_class):
        X_result[np.where(labels == i), 0] = Hex_to_RGB(hex_color_dict[i])[0]
        X_result[np.where(labels == i), 1] = Hex_to_RGB(hex_color_dict[i])[1]
        X_result[np.where(labels == i), 2] = Hex_to_RGB(hex_color_dict[i])[2]

    X_result = np.reshape(X_result, (row, col, 3))

    return X_result

def Cal_INDEX(x):

    x=x.astype('float')

    x[x>10000]=10000
    x[x<0]=0

    R = x[:, :, 3]
    G = x[:, :, 2]
    Nir = x[:, :, 7]  # TM4
    Mir = x[:, :, 10]  # TM5
    SWir= x[:,:,11]

    NDWI = (G - Nir) / (G + Nir)
    NDVI = (Nir - R) / (Nir + R)
    NDSI = (Mir-Nir) / (Mir+Nir)
    NBI = R * SWir / Nir

    return NDWI,NDVI, NDSI, NBI

class Evaluator(object):
    def __init__(self, num_class):
        self.num_class = num_class
        self.confusion_matrix = np.zeros((self.num_class,)*2)
        # matrix shape(num_class, num_class) with elements 0 in our match. it will be 4*4

    def Kappa(self):        
        
        xsum = np.sum(self.confusion_matrix, axis=1)  # sum by row
        ysum = np.sum(self.confusion_matrix, axis=0)  # sum by column
       
        Pe = np.sum(ysum*xsum)*1.0/(self.confusion_matrix.sum()**2)
        P0 = np.diag(self.confusion_matrix).sum() / self.confusion_matrix.sum()  # predict right / all the data
        cohens_coefficient = (P0-Pe)/(1-Pe)

        return cohens_coefficient
            
    def ProducerA(self):
        #
        return np.diag(self.confusion_matrix) / np.sum(self.confusion_matrix, axis=1)

    def UserA(self):
        #
        return np.diag(self.confusion_matrix) / np.sum(self.confusion_matrix, axis=0)


    def Pixel_Accuracy(self):
        Acc = np.diag(self.confusion_matrix).sum() / self.confusion_matrix.sum()
        return Acc

    def val_Pixel_Accuracy_Class(self):
        Acc = np.diag(self.confusion_matrix) / self.confusion_matrix.sum(axis=1)
        # each pred right class is in diag. sum by row is the count of corresponding class
#         index=np.array([0,1])
#         Acc=Acc[index]
        #Acc[-1]=90
        Acc = np.nanmean(Acc) #
        return Acc

    def pre_Pixel_Accuracy_Class(self):
        Acc = np.diag(self.confusion_matrix) / self.confusion_matrix.sum(axis=1)
        # each pred right class is in diag. sum by row is the count of corresponding class
#         index=np.array([0,1])
#         Acc=Acc[index]
        #Acc[-1]=90
        Acc = np.nanmean(Acc) #
        return Acc

    def Mean_Intersection_over_Union(self):
        MIoU = np.diag(self.confusion_matrix) / (
                    np.sum(self.confusion_matrix, axis=1) + np.sum(self.confusion_matrix, axis=0) -
                    np.diag(self.confusion_matrix))
        MIoU = np.nanmean(MIoU)
        return MIoU

    def Frequency_Weighted_Intersection_over_Union(self):
        freq = np.sum(self.confusion_matrix, axis=1) / np.sum(self.confusion_matrix)
        iu = np.diag(self.confusion_matrix) / (
                    np.sum(self.confusion_matrix, axis=1) + np.sum(self.confusion_matrix, axis=0) -
                    np.diag(self.confusion_matrix))

        FWIoU = (freq[freq > 0] * iu[freq > 0]).sum()
        return FWIoU

    def _generate_matrix(self, gt_image, pre_image):
        # gt_image = batch_size*256*256   pre_image = batch_size*256*256
        mask = (gt_image >= 0) & (gt_image < self.num_class)  # valid in mask show True, ignored in mask show False
        label = self.num_class * gt_image[mask].astype('int') + pre_image[mask]
        # gt_image[mask] : find out valid pixels. elements with 0,1,2,3 , so label range in  0-15
        count = np.bincount(label, minlength=self.num_class**2)
        # [0, 1, 2, 3,  confusion_matrix like this:
        #  4, 5, 6, 7,  and if the element is on the diagonal, it means predict the right class.
        #  8, 9, 10,11, row means the real label, column means pred label
        #  12,13,14,15]
        # return a array [a,b....], each letters holds the count of a class and map to class0, class1...
        confusion_matrix = count.reshape(self.num_class, self.num_class)
        return confusion_matrix

    def add_batch(self, gt_image, pre_image):
        assert gt_image.shape == pre_image.shape
        self.confusion_matrix += self._generate_matrix(gt_image, pre_image)

    def reset(self):
        self.confusion_matrix = np.zeros((self.num_class,) * 2)

In [26]:
from scipy.misc import imsave
from PIL import Image

In [27]:
base_dir = '/data/PublicData/DF2020/val/'
s1_pre, s2_pre, lc_pre=load_data(base_dir)

100%|██████████| 986/986 [00:00<00:00, 198406.44it/s]


load model

In [28]:
clf=joblib.load('xgb_val_sa_shrb.pkl') 

In [29]:
evaluator=Evaluator(10)
main_dir='/data/di.wang/ordinary/23DCNN/DataFusion_2020_new/preval/xgb_filter_sa_shrb/'

outputdir=main_dir+'output/'
visdir=main_dir+'vis/'

if not os.path.exists(outputdir):
    os.makedirs(outputdir)

if not os.path.exists(visdir):
    os.makedirs(visdir)

In [30]:
for i in tqdm(range(s1_pre.shape[0])):
    s1_name=s1_pre[i]
    s2_name=s2_pre[i]
    y_name=lc_pre[i]
    x1,x2=read_data(s1_name,s2_name)
    x1=x1.transpose(1,2,0)#h,w,c
    x2=x2.transpose(1,2,0)
    x2_tmp=x2.copy()
    x2=x2[:,:,[1,2,3,4,5,6,7,10,11,12]]#remove b1,9,10
    
    x1,x2=clean(x1,x2)
    
    with rasterio.open(y_name) as patch:
        y = patch.read(list(range(1, 5)))
    y = process_label(y)
    y = filter_label(x2_tmp, y)
    x1, x2 = norm(x1, x2)
    
    x1=x1.reshape(-1,2)
    x2=x2.reshape(-1,10)
    x=np.concatenate((x1,x2),axis=1)
    y_pre=clf.predict(x).reshape(256,256)
    
    # postprocessing
    NDWI,NDVI,NDSI,NBI=Cal_INDEX(x2_tmp)
    im = np.uint8(y_pre + 1)
    
    y_tmp=y[0].copy()
    im_tmp=im.copy()
    y_pre_tmp=y_pre.copy()
    
    # grassland
    
#     im[np.where((NDVI>0.4) & (NDVI<0.6)& (y_tmp==3) &(np.sum(im_tmp==10)<2000))]=4
#     y_pre[np.where((NDVI>0.4) & (NDVI<0.6)&(y_tmp==3) &(np.sum(y_pre_tmp==9)<2000))]=3
    
#     # wetland
    
#     im[np.where((NDVI>0.6) & (NDVI<0.75)& ((im_tmp==4)|(im_tmp==1))&(np.sum(im_tmp==10)>2000))]=5
#     y_pre[np.where((NDVI>0.6) & (NDVI<0.75)& ((im_tmp==4)|(im_tmp==1)) &(np.sum(y_pre_tmp==9)>2000))]=4
    
#     # barren
    
#     im[np.where((NBI > 750) &(NDVI<0.4)& (NDVI>0) & (y_tmp!=5) & (y_tmp!=6) &((im_tmp==1) | (im_tmp==4) | (im_tmp==2)))] = 9
#     y_pre[np.where((NBI > 750) &(NDVI<0.4) & (NDVI>0) & (y_tmp!=5) & (y_tmp!=6)& ((y_pre_tmp==0) | (y_pre_tmp==3) | (y_pre_tmp==1)))] = 8
    
    evaluator.add_batch(y, y_pre[np.newaxis, :, :])
    
    filename=os.path.basename(y_name)
    former = filename.split('lc')[0]
    latter = filename.split('lc')[1]
    
    imsave(outputdir+former+'dfc'+latter,im)
    im_rgb = Image.fromarray(np.uint8(DrawResult(y_pre.reshape(-1), 256, 256)))
    im_rgb.save(visdir + former+'dfc'+latter[:-4] + '_vis.png')

  0%|          | 0/986 [00:00<?, ?it/s]

[16:58:16] WARNING: /workspace/include/xgboost/generic_parameters.h:36: 
n_gpus: 
	Deprecated. Single process multi-GPU training is no longer supported.
	Please switch to distributed training with one process per GPU.
	This can be done using Dask or Spark.  See documentation for details.


/data/di.wang/.conda/envs/pytorch/lib/python3.5/site-packages/ipykernel_launcher.py:53: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
100%|██████████| 986/986 [14:22<00:00,  1.14it/s]


In [31]:
AA = evaluator.pre_Pixel_Accuracy_Class()

print('AVERAGE ACCURACY of val DATASET: {}'.format(AA))

print('CONFUSION MATRIX')

print(evaluator.confusion_matrix)

print('ACCURACY IN EACH CLASSES:',np.diag(evaluator.confusion_matrix) / evaluator.confusion_matrix.sum(axis=1))

print('Prediction finished!')

AVERAGE ACCURACY of val DATASET: 0.8536393893844794
CONFUSION MATRIX
[[2.7735480e+06 0.0000000e+00 0.0000000e+00 0.0000000e+00 9.8600000e+02
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 4.3074800e+05 0.0000000e+00 1.4594200e+05 4.4000000e+01
  4.7560000e+04 8.5040000e+03 0.0000000e+00 4.9410000e+03 2.0000000e+01]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [2.8000000e+01 8.1360000e+04 0.0000000e+00 1.5632350e+06 2.0100000e+03
  3.0959300e+05 1.4074400e+05 0.0000000e+00 1.9313000e+04 1.6832000e+04]
 [1.0340000e+03 2.2900000e+02 0.0000000e+00 1.1291000e+04 1.9335440e+06
  2.4760000e+03 1.0240000e+04 0.0000000e+00 1.4320000e+03 7.3650000e+03]
 [1.4000000e+01 1.0249000e+04 0.0000000e+00 1.3655100e+05 2.8600000e+02
  3.1404870e+06 1.2036800e+05 0.0000000e+00 5.0240000e+03 4.0300000e+02]
 [3.0000000e+01 3.9880000e+03 0.0000000e+00 9.5826000e+04 7.6

/data/di.wang/.conda/envs/pytorch/lib/python3.5/site-packages/ipykernel_launcher.py:83: RuntimeWarning: invalid value encountered in true_divide
/data/di.wang/.conda/envs/pytorch/lib/python3.5/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
